## Network distribution


* Define the linear model with mathematical notation, write it with math formulas 


This is a network distribution with intermediates nodes, the origin nodes are four and named by 1,2,3,4, the three intermediates nodes are named by 5,6,7, the four destinations nodes are named 8, 9, 10, 11. 


| Destinations | Requirements |
| --- | --- |
| Node 8 | 90 |
| Node 9 | 150 | 
| Node 10| 110 | 
| Node 11 | 70 |


The origin nodes:

|  Origin | Capacity |
| --- | --- |
| Node 1  | 150 |
| Node 2  | 115 |
| Node 3  | 110 |
| Node 4  | 65 |


Each intermediate nodes have is own capacity

| Intermediate nodes   | Capacity |
| --- | --- |
| Node 5  | 250 |
| Node 6 | 120 |
| Node 7 | 200 |


Let be $X_{ij}$ the amount of the product you want to transport from the origin $i$ towars destination $j$ in this case, the intermediate nodes and let be $Y_{ij}$ the amount of the product you want to transport from the intermediate nodes $i$ to the final destination $j$, all the index must be 1, 2, 3, 4. Let be $c_{ij}$ the cost for transport the product from the node $i$ to the node $j$ or from the intermediate node $i$ to the destinatio $j$. $c_{ij}$ is the cost associate to the transport path between the origin to destination. 
 
Always want to minimize the cost of transporting the product:

$$
min Z = c_{11} X_{11} + c_{12} X_{12} + c_{13} X_{13} + c_{14} X_{14} + c_{21} Y_{21}+ c_{22} Y_{22} + c_{23} Y_{23} + c_{24} Y_{24} + \\ c_{31} Y_{31} + c_{32} Y_{32} + c_{33} Y_{33} + c_{34} Y_{34} + c_{41} Y_{41} + c_{42} Y_{42}+ c_{43} Y_{43} + c_{44} Y_{44}
$$

This problem must be constrain in the offer, demand, the capacity of the intermediate nodes y must be positives values all what we got about this variables. This means:
1. Constrain in the offer: the total amount of the product transported from the origin can't exceed the available offer in the origin, this is: 

$$
X_{11} + X_{12} + X_{13}+ X_{14} \leq D
$$
2. Constrain the demand: the amount of product transported to each destination must satisfied their demand:

\begin{gather}
X_{11} + Y_{21} + Y_{31} + Y_{41} = D_1  \\
X_{12} + Y_{22} + Y_{32} + Y_{42} = D_2  \\
X_{13} + Y_{23} + Y_{33} + Y_{43} = D_3  \\
X_{14} + Y_{24} + Y_{34} + Y_{44} = D_4  \\
\end{gather}
3. Intermediate nodes constrain: the amount of the product transported through each nodes can't exceed their transport capacity:

\begin{gather}
X_{11} + X_{12} + X_{13} + X_{14} \leq C_1\\
Y_{21} + Y_{22} + Y_{23} + Y_{24} \leq C_2\\
Y_{31} + Y_{32} + Y_{33} + Y_{34} \leq C_3\\
\end{gather}
4. Non-negative constrain: the amount of the product transported must be positive:
\begin{gather}
X_{ij} \geq 0 \\
Y_{ij} \geq 0
\end{gather}

$D$ is the available offer in the origin, from $D_1$ to $D_4$ are the destination demands, from $C_1$ to  $C_3$ are the intermediates nodes capacity

* To solve this network problem, we need to follow the next steps:

    1. Import the libraries or modules we need.
    2. Create the model.
    3. Define the parameters of the problem.
    4. Define the decision variables.
    5. Define the objective variable ($Z$).
    6. Add the constrains
    7. Solve the model.
    8. Show the obtained solution.
    
I'll go use docplex library designed by IBM.

In [1]:
# 1st step: import librarys or modules
from docplex.mp.model import Model 

In [2]:
# 2nd Step: create the model to solve
model = Model(name='Network_distribution') # Create a model

In [3]:
# 3rd Step: define the parameters
origin = [1,2,3,4] # Origin nodes
intermediate_nodes = [5,6,7] # Intermediate nodes
destination = [8,9,10,11] # destination nodes

# All nodes capacity: 
trans_capacity = {(1, 5): 100, (1, 6): 150, (1, 7): 200, (2, 5): 200, (2, 6): 250, (2, 7): 200,
                  (3, 5): 300, (3, 6): 200, (3, 7): 200, (4, 5): 150, (4, 6): 400, (4, 7): 200,
                  (5, 8): 100, (5, 9): 200, (5, 10):250, (5,11): 500, (6, 8): 150, (6, 9): 250,
                  (6, 10):200, (6, 11):200, (7, 8): 200, (7, 9): 300, (7, 10):130, (7, 11):200}
# Cost transporting 
trans_cost = {(1, 5): 5, (1, 6): 2, (1, 7): 7 , (2, 5):3, (2, 6): 6, (2, 7): 6,
              (3, 5): 6, (3, 6): 1, (3, 7): 2, (4, 5): 4, (4, 6): 3, (4, 7): 6,
              (5, 8): 5, (5, 9): 6, (5, 10):8, (5,11): 5, (6, 8): 6, (6, 9): 5,
              (6, 10):5, (6, 11):2, (7, 8): 4, (7, 9): 3, (7, 10):3, (7, 11):9}
#Destination needs
destination_needs = {8: 90, 9: 150, 10:110, 11: 70}

In [4]:
# 4th step: Decision variables

# Variables from origin to intermediate nodes
trans_vars  = { (i,j): model.continuous_var(lb=0, name=f'trans_{i}_{j}')
               for i in origin for j in intermediate_nodes
}
# Variables from intermediate nodes to destination
trans_vars2 = { (i,j): model.continuous_var(lb=0, name=f'trans2_{i}_{j}')
               for i in intermediate_nodes for j in destination
}


In [5]:
# 5th Objective function  

model.minimize(model.sum(trans_cost[i, j] * trans_vars[i, j] for i in origin for j in intermediate_nodes) +
               model.sum(trans_cost[i, j] * trans_vars2[i, j] for i in intermediate_nodes for j in destination))

In [6]:
# 6th add constrain:

# Constrain: the amount of the product sent to each node must be greater or equal to needed:
for k in destination:
    model.add_constraint(model.sum(trans_vars2[j, k] for j in intermediate_nodes) >= destination_needs[k])


# Constrain: the amount of the product sent from each origin and intermediate nodes cannot exceed their capacity:
for i in origin:
    for j in intermediate_nodes:
        model.add_constraint(trans_vars[i, j] <= trans_capacity[(i, j)])
for i in intermediate_nodes:
    for j in destination:
        model.add_constraint(trans_vars2[i, j] <= trans_capacity[(i, j)])

# Constrain: the total amount of the product sent from each origin and each intermediate nodes must be equal to the total amount
#            received by each destination node.

for k in destination:
    model.add_constraint(model.sum(trans_vars2[j, k] for j in intermediate_nodes) ==
                         model.sum(trans_vars[i, j] for i in origin for j in intermediate_nodes if (i, j) in trans_capacity))

In [7]:
# 7th Solve 
solution = model.solve()

# 8th show the obtained solution: 
for var in trans_vars:
    print(f'{var}: {trans_vars[var].solution_value}')
    
for var in trans_vars2:
    print(f'{var}: {trans_vars2[var].solution_value}')


(1, 5): 0
(1, 6): 0
(1, 7): 0
(2, 5): 0
(2, 6): 0
(2, 7): 0
(3, 5): 0
(3, 6): 150.0
(3, 7): 0
(4, 5): 0
(4, 6): 0
(4, 7): 0
(5, 8): 0
(5, 9): 0
(5, 10): 0
(5, 11): 0
(6, 8): 0
(6, 9): 0
(6, 10): 20.0
(6, 11): 150.0
(7, 8): 150.0
(7, 9): 150.0
(7, 10): 130.0
(7, 11): 0


c. Plotting the solution

In [8]:
import matplotlib.pyplot as plt

# Coordinates of the locations
coords = {1: (0, 0), 2: (0, 5), 3: (0, 10), 4: (0, 15), 5: (5, 2.5), 6: (5, 7.5), 7: (5, 12.5), 8: (10, 3.75),
          9: (10, 8.75), 10: (10, 13.75), 11: (10, 18.75)}

# Plot the locations
plt.figure(figsize=(8, 8))
for i, loc in coords.items():
    if i in [1, 2, 3, 4]:
        plt.plot(loc[0], loc[1], 'o', markersize=10, label=f'Origin {i}')
    elif i in [5, 6, 7]:
        plt.plot(loc[0], loc[1], '^', markersize=10, label=f'Intermediate node {i}')
    else:
        plt.plot(loc[0], loc[1], 's', markersize=10, label=f'Destination {i}')
plt.legend()

# Plot the optimal routes
for i, j in trans_vars:
    if trans_vars[i, j].solution_value > 0:
        plt.plot([coords[i][0], coords[j][0]], [coords[i][1], coords[j][1]], 'k--')
for i, j in trans_vars2:
    if trans_vars2[i, j].solution_value > 0:
        plt.plot([coords[i][0], coords[j][0]], [coords[i][1], coords[j][1]], 'k--')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()


<Figure size 800x800 with 1 Axes>

d) Organize the solution in a table:

In [9]:
import pandas as pd

# Extract solution information
total_cost = solution.get_objective_value()
transport_df = pd.DataFrame.from_dict(trans_vars, orient='index', columns=['Variable'])
transport_df['Solution'] = transport_df['Variable'].apply(lambda x: x.solution_value)
transport_df = transport_df[transport_df['Solution'] > 0]

# Create summary table
summary_table = pd.DataFrame(columns=['Origin', 'Destination', 'Amount'])
for index, row in transport_df.iterrows():
    source = index[0]
    destination = index[1]
    amount = row['Solution']
    summary_table = summary_table.append({'Origin': source, 'Destination': destination, 'Amount': amount}, ignore_index=True)

# Print summary table
print(summary_table)

   Origin  Destination  Amount
0     3.0          6.0   150.0
